# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [16]:
import cv2
from ultralytics import YOLO
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

model = YOLO('/content/YOLO_Model.pt')
dict_classes = model.model.names

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [17]:
video_path = '/content/YOLO_Video.mp4'
video = cv2.VideoCapture(video_path)

In [18]:
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = video.get(cv2.CAP_PROP_FPS)

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [19]:
output_path = '/content/YOLO_Video.mp4'
VIDEO_CODEC = "MP4V"
output_video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*VIDEO_CODEC), fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

In [20]:
ret, frame = video.read()

In [21]:
x1, y1, x2, y2 = 100, 200, 300, 400
cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)



### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [24]:
# Define ROI coordinates
top_x1, top_y1, top_x2, top_y2 = 10, 0, 1270, 100
bottom_x1, bottom_y1, bottom_x2, bottom_y2 = 10, 710, 1270, 800
left_x1, left_y1, left_x2, left_y2 = 0, 100, 100, 710
right_x1, right_y1, right_x2, right_y2 = 1170, 100, 1280, 710

top_count, bottom_count, left_count, right_count = 0, 0, 0, 0

while True:
    ret, frame = video.read()
    if not ret:
        break

    results = model.predict(frame)

    for result in results:
        detections = result.boxes
        for detection in detections:
            bbox = detection.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, bbox)
            cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

            conf = round(detection.conf.item(), 2)
            cls = int(detection.cls.item())

            # Count vehicles leaving each side
            if top_x1 < cx < top_x2 and top_y1 < cy < top_y2:
                top_count += 1
            if bottom_x1 < cx < bottom_x2 and bottom_y1 < cy < bottom_y2:
                bottom_count += 1
            if left_x1 < cx < left_x2 and left_y1 < cy < left_y2:
                left_count += 1
            if right_x1 < cx < right_x2 and right_y1 < cy < right_y2:
                right_count += 1

            # Annotate frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            label = f"{conf} {cls}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw ROI rectangles
    cv2.rectangle(frame, (top_x1, top_y1), (top_x2, top_y2), (0, 255, 0), 2)
    cv2.rectangle(frame, (bottom_x1, bottom_y1), (bottom_x2, bottom_y2), (0, 255, 0), 2)
    cv2.rectangle(frame, (left_x1, left_y1), (left_x2, left_y2), (0, 255, 0), 2)
    cv2.rectangle(frame, (right_x1, right_y1), (right_x2, right_y2), (0, 255, 0), 2)


   # Display counts
    cv2.putText(frame, f"Top: {top_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Bottom: {bottom_count}", (10, 770), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Left: {left_count}", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Right: {right_count}", (1150, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

In [25]:
from google.colab.patches import cv2_imshow

In [26]:
print(dict_classes)

{0: 'bus', 1: 'car', 2: 'truck', 3: 'van'}


In [27]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [28]:
output_video.write(frame)

In [29]:
video.release()
output_video.release()

## Save and Submit Your Work

Submit both the notebook and the output video